# GNN stream adj,fea and weight matrix multiplication with AXI master interfaces on PYNQ

## Import and download the HLS Overlay

In [845]:
#import os
#base_dir=os.getcwd()
#filename="/home/xilinx/pynq/overlays/gnn/gnn.bit";
#bitname=os.path.join(base_dir, filename)
#print(bitname)
#import torch

In [846]:
from pynq import Overlay
ol = Overlay("gnn_all.bit")

In [847]:
print("DATASET selection")

DATASET selection


In [562]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/cora_weights.txt'
N_adj = 2708  # number of nodes
M_fea = 1433  # number of input features
P_w = 16  # number of features in the hidden layer
NNZ_adj = 13264  # number of non-zero values of adjacency
NNZ_fea = 49216  # number of non-zero values of feature
ADJ_THREADS = 1

In [848]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/citeseer_weights.txt'
N_adj = 3327  # number of nodes
M_fea = 3703  # number of input features
P_w = 16  # number of features in the hidden layer
NNZ_adj = 12431  # number of non-zero values of adjacency
NNZ_fea = 105165  # number of non-zero values of feature
ADJ_THREADS = 1

In [263]:
adj_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn_all/matrices/pubmed_weights.txt'
N_adj = 19717  # number of nodes
M_fea = 500  # number of input features
P_w = 16  # number of features in the hidden layer
NNZ_adj = 108365  # number of non-zero values of adjacency
NNZ_fea = 988031  # number of non-zero values of feature
ADJ_THREADS = 1

adj_name = '/home/xilinx/jupyter_notebooks/gnn/matrices/test_adj.txt'
fea_name = '/home/xilinx/jupyter_notebooks/gnn/matrices/test_feat.txt'
w_name = '/home/xilinx/jupyter_notebooks/gnn/matrices/test_weights.txt'
N_adj = 4  # number of nodes
M_fea = 4  # number of input features
P_w = 2  # number of features in the hidden layer
NNZ_adj = 4  # number of non-zero values of adjacency
NNZ_fea = 2  # number of non-zero values of feature
FRAC_BITS_IN = 14
FRAC_BITS_OUT = 14
DATA_WIDTH = 16

## Examine the design 

In [849]:
ol.ip_dict

{'axi_intc_0': {'fullpath': 'axi_intc_0',
  'type': 'xilinx.com:ip:axi_intc:4.1',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 2684354560,
  'mem_id': 's_axi',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {'intr': {'controller': 'axi_intc_0',
    'index': 0,
    'fullpath': 'axi_intc_0/intr'}},
  'parameters': {'C_FAMILY': 'zynquplus',
   'C_INSTANCE': 'design_1_axi_intc_0_0',
   'C_S_AXI_ADDR_WIDTH': '9',
   'C_S_AXI_DATA_WIDTH': '32',
   'C_NUM_INTR_INPUTS': '1',
   'C_NUM_SW_INTR': '0',
   'C_KIND_OF_INTR': '0xfffffffe',
   'C_KIND_OF_EDGE': '0xFFFFFFFF',
   'C_KIND_OF_LVL': '0xFFFFFFFF',
   'C_ASYNC_INTR': '0xFFFFFFFE',
   'C_NUM_SYNC_FF': '2',
   'C_ADDR_WIDTH': '32',
   'C_IVAR_RESET_VALUE': '0x0000000000000010',
   'C_ENABLE_ASYNC': '0',
   'C_HAS_IPR': '1',
   'C_HAS_SIE': '1',
   'C_HAS_CIE': '1',
   'C_HAS_IVR': '1',
   'C_HAS_ILR': '0',
   'C_IRQ_IS_LEVEL': '1',
   'C_IRQ_ACTIVE': '0x1',
   'C_DISABLE_SYNCHRONIZERS': '0',
   'C_MB_CLK_NO

## Create an alias for the IP

In [850]:
my_ip = ol.mmult_top_0
#my_monitor = ol.my_monitor_0

## Check the register map for the HLS IP

You should see the registers that we need to write values to:

* CTRL - the control register
* A_offset_1 - the lower 32-bits of the start address for the A matrix
* B_offset_1 - the lower 32-bits of the start address for the B matrix
* C_offset_1 - the lower 32-bits of the start address for the C matrix

In [851]:
my_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  gemm_mode = Register(gemm_mode=write-only, RESERVED=write-only),
  quantized_multiplier_offset_1 = Register(quantized_multiplier_offset=write-only),
  quantized_multiplier_offset_2 = Register(quantized_multiplier_offset=write-only),
  shift_offset_1 = Register(shift_offset=write-only),
  shift_offset_2 = Register(shift_offset=write-only),
  bias_offset_1 = Register(bias_offset=write-only),
  bias_offset_2 = Register(bias_offset=write-only),
  bias_count = Register(bias_count=write-only),
  profiling_offset_1 = Register(profiling_offset=write-only),
  profiling_offset_2 = Register(profiling_offset=write-only),
  zero_point_lhs = Register(zero_point_lhs=write-only, R

In [852]:
my_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  gemm_mode = Register(gemm_mode=write-only, RESERVED=write-only),
  quantized_multiplier_offset_1 = Register(quantized_multiplier_offset=write-only),
  quantized_multiplier_offset_2 = Register(quantized_multiplier_offset=write-only),
  shift_offset_1 = Register(shift_offset=write-only),
  shift_offset_2 = Register(shift_offset=write-only),
  bias_offset_1 = Register(bias_offset=write-only),
  bias_offset_2 = Register(bias_offset=write-only),
  bias_count = Register(bias_count=write-only),
  profiling_offset_1 = Register(profiling_offset=write-only),
  profiling_offset_2 = Register(profiling_offset=write-only),
  zero_point_lhs = Register(zero_point_lhs=write-only, R

In [853]:
print("Allocate some memory")

##The start address for this memory will be passed to the HLS IP. The HLS IP will read data from this memory and write results back to the same locations. 

##We can initialise the memory 

Allocate some memory


In [854]:
from pynq import allocate
import numpy as np 
quantized_multiplier_buffer  = allocate(1024, dtype=np.int32)
bias_buffer  = allocate(1024, dtype=np.int32)
shift_buffer  = allocate(1024, dtype=np.int32)
profiling_buffer  = allocate(16, dtype=np.int64)
rowPtr_fea_buffer = allocate(N_adj+1, dtype=np.int32)
columnIndex_fea_buffer = allocate(NNZ_fea, dtype=np.int32)
values_fea_buffer = allocate(NNZ_fea, dtype=np.float16)
rowPtr_adj_buffer = allocate(N_adj+1, dtype=np.int32)
columnIndex_adj_buffer = allocate(NNZ_adj, dtype=np.int32)
values_adj_buffer = allocate(NNZ_adj, dtype=np.float16)
B_buffer = allocate(shape=(P_w,M_fea), dtype=np.float16)
#B_buffer = allocate((M_fea*P_w), dtype=np.float32)
#D_buffer = allocate((N_adj*P_w), dtype=np.float32)
D_buffer = allocate(shape=(N_adj,P_w), dtype=np.float16)

In [855]:
## Initialize the D and B buffers

print("Load weight data")

Load weight data


In [856]:
fin = open(w_name, 'rb')
#w_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float32)
w = np.genfromtxt(fin, delimiter=',', dtype=np.float16)
for i in range(M_fea): 
    for j in range(P_w):
            B_buffer[j][i] = w[i][j]
            #B_buffer[i*P_w+j] = w[i][j] * (1 << FRAC_BITS_IN)
            #B_buffer[i*P_w+j] = 255

In [857]:
#B_buffer=B_buffer.transpose() #we transpose weights so hardware can read column as contiguos data

In [858]:
B_buffer

PynqBuffer([[ 0.01738  ,  0.03238  , -0.000407 , ..., -0.001362 ,
             -0.       , -0.000456 ],
            [-0.01135  , -0.0333   , -0.000504 , ..., -0.001169 ,
             -0.       , -0.006878 ],
            [-0.02078  ,  0.0268   ,  0.000892 , ..., -0.003405 ,
              0.       , -0.0085   ],
            ...,
            [-0.018    ,  0.01904  ,  0.001408 , ...,  0.001011 ,
              0.       ,  0.007812 ],
            [ 0.01994  ,  0.05753  ,  0.0002897, ..., -0.00202  ,
             -0.       , -0.002542 ],
            [-0.001315 , -0.02975  , -0.0001557, ...,  0.003527 ,
              0.       ,  0.010315 ]], dtype=float16)

In [859]:
for i in range(N_adj):
    for j in range(P_w):
        D_buffer[i][j] = 1

In [860]:
np.amin(B_buffer)

PynqBuffer(-0.1556, dtype=float16)

In [861]:
np.amax(B_buffer)

PynqBuffer(0.265, dtype=float16)

for i in range(32):
    for j in range(32):
        B_buffer[i][j] = 1
for i in range(32):
    for j in range(32):
        D_buffer[i][j] = 0

In [862]:
print("Read sparse data")

Read sparse data


## Read Sparse data



with open('/home/xilinx/jupyter_notebooks/gnn/test_000_adj.csv', 'rb') as fin:
    rowPtr_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    for i in range(32):
        rowPtr_adj_buffer[i] = rowPtr_adj_row[i]
    columnIndex_adj_buffer = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    values_adj_buffer = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)

In [863]:
with open(adj_name, 'rb') as fin:
    rowPtr_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #print(rowPtr_adj_row[3327])
    #for i in range(N_adj+1):
        #print(rowPtr_adj_row[i])
        #rowPtr_adj_buffer[i] = rowPtr_adj_row[i]
    rowPtr_adj_buffer[:] = rowPtr_adj_row[:]
    columnIndex_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(NNZ_adj):
        #columnIndex_adj_buffer[i] = columnIndex_adj_row[i]
    columnIndex_adj_buffer[:] = columnIndex_adj_row[:]
    values_adj_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float16)
    #for i in range(NNZ_adj):
    values_adj_buffer[:] = values_adj_row[:]
    #values_adj_buffer[:] = values_adj_row[:]
        #values_adj_buffer[i] = 255

In [864]:
columnIndex_adj_buffer

PynqBuffer([   0,  628,    1, ..., 3325,   33, 3326], dtype=int32)

In [865]:
np.amax(values_adj_row)

1.0

In [866]:
values_adj_buffer

PynqBuffer([0.5   , 0.5   , 0.1666, ..., 0.5   , 0.2886, 0.5   ],
           dtype=float16)

In [867]:
with open(fea_name, 'rb') as fin:
    rowPtr_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(N_adj+1):
        #rowPtr_fea_buffer[i] = rowPtr_fea_row[i]
        #print(rowPtr_fea_buffer[i])
    rowPtr_fea_buffer[:] = rowPtr_fea_row[:]
    columnIndex_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    #for i in range(NNZ_fea):
    #    columnIndex_fea_buffer[i] = columnIndex_fea_row[i]
    columnIndex_fea_buffer[:] = columnIndex_fea_row[:]
    #columnIndex_fea_buffer = columnIndex_fea_row
    values_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float16)
    #for i in range(NNZ_fea):
        #values_fea_buffer[i] = 0
    #values_fea_buffer[:] = values_fea_row[:]
    values_fea_buffer[:] = values_fea_row[:]

In [868]:
rowPtr_fea_buffer

PynqBuffer([     0,     31,     64, ..., 105103, 105139, 105165],
           dtype=int32)

In [869]:
np.amax(values_fea_row)

1.0

with open(fea_name, 'rb') as fin:
    rowPtr_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    rowPtr_fea_buffer[:] = rowPtr_fea_row
    columnIndex_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.int32)
    columnIndex_fea_buffer[:] = columnIndex_fea_row[0:N_adj]
    values_fea_row = np.genfromtxt(fin, max_rows=1, delimiter=',', dtype=np.float32)
    values_fea_buffer[:] = values_fea_row * (1 << FRAC_BITS)
        

In [870]:
 values_adj_buffer

PynqBuffer([0.5   , 0.5   , 0.1666, ..., 0.5   , 0.2886, 0.5   ],
           dtype=float16)

In [871]:
print("rowPtr_fea_buffer physical address {}".format(hex(rowPtr_fea_buffer.physical_address)))
print("columnIndex_fea_buffer physical address {}".format(hex(columnIndex_fea_buffer.physical_address)))
print("values_fea_buffer physical address {}".format(hex(values_fea_buffer.physical_address)))
print("rowPtr_adj_buffer physical address {}".format(hex(rowPtr_adj_buffer.physical_address)))
print("columnIndex_adj_buffer physical address {}".format(hex(columnIndex_adj_buffer.physical_address)))
print("quantized_multiplier physical address {}".format(hex(quantized_multiplier_buffer.physical_address)))
print("bias physical address {}".format(hex(bias_buffer.physical_address)))
print("shift physical address {}".format(hex(shift_buffer.physical_address)))
print("B_buffer physical address {}".format(hex(B_buffer.physical_address)))
print("D_buffer physical address {}".format(hex(D_buffer.physical_address)))
print("profiling physical address {}".format(hex(profiling_buffer.physical_address)))

rowPtr_fea_buffer physical address 0x7a06c000
columnIndex_fea_buffer physical address 0x7db00000
values_fea_buffer physical address 0x7db80000
rowPtr_adj_buffer physical address 0x7a1c4000
columnIndex_adj_buffer physical address 0x7a270000
quantized_multiplier physical address 0x739fc000
bias physical address 0x739fd000
shift physical address 0x739fe000
B_buffer physical address 0x7dae0000
D_buffer physical address 0x7dbc0000
profiling physical address 0x739ff000


In [872]:
#columnIndex_adj_buffer

In [873]:
 #rowPtr_adj_buffer[:] = rowPtr_adj_row

In [874]:
 #rowPtr_adj_buffer[:]=0 

In [875]:
 #rowPtr_adj_buffer[1:] = 1

In [876]:
print("configure IP")

configure IP


## Configure the IP

Write the values of the memory address for all the buffers.
The run kernel starts the IP block and checks the done bit to see when it goes high. 

In [877]:
my_ip.register_map.B_offset_1 = B_buffer.physical_address
my_ip.register_map.D1_offset_1 = D_buffer.physical_address
my_ip.register_map.D2_offset_1 = D_buffer.physical_address+P_w*N_adj/ADJ_THREADS
my_ip.register_map.D3_offset_1 = D_buffer.physical_address+P_w*2*N_adj/ADJ_THREADS
my_ip.register_map.D4_offset_1 = D_buffer.physical_address+P_w*3*N_adj/ADJ_THREADS
my_ip.register_map.rowPtr_fea1_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea2_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea3_offset_1 = rowPtr_fea_buffer.physical_address
my_ip.register_map.rowPtr_fea4_offset_1 = rowPtr_fea_buffer.physical_address
#my_ip.register_map.rowPtr_fea_offset_1 = 0
my_ip.register_map.columnIndex_fea1_offset_1 =columnIndex_fea_buffer.physical_address
my_ip.register_map.columnIndex_fea2_offset_1 =columnIndex_fea_buffer.physical_address 
my_ip.register_map.columnIndex_fea3_offset_1 =columnIndex_fea_buffer.physical_address 
my_ip.register_map.columnIndex_fea4_offset_1 =columnIndex_fea_buffer.physical_address 
#my_ip.register_map.columnIndex_fea_offset_1 =0
my_ip.register_map.values_fea1_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea2_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea3_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.values_fea4_offset_1 = values_fea_buffer.physical_address 
my_ip.register_map.rowPtr_adj1_offset_1 = rowPtr_adj_buffer.physical_address 
my_ip.register_map.rowPtr_adj2_offset_1 = rowPtr_adj_buffer.physical_address 
my_ip.register_map.rowPtr_adj3_offset_1 = rowPtr_adj_buffer.physical_address
my_ip.register_map.rowPtr_adj4_offset_1 = rowPtr_adj_buffer.physical_address
#my_ip.register_map.rowPtr_adj_offset_1 = 0
my_ip.register_map.columnIndex_adj1_offset_1 = columnIndex_adj_buffer.physical_address 
my_ip.register_map.columnIndex_adj2_offset_1 = columnIndex_adj_buffer.physical_address
my_ip.register_map.columnIndex_adj3_offset_1 = columnIndex_adj_buffer.physical_address 
my_ip.register_map.columnIndex_adj4_offset_1 = columnIndex_adj_buffer.physical_address 
#my_ip.register_map.columnIndex_adj_offset_1 = 0
my_ip.register_map.values_adj1_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj2_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj3_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.values_adj4_offset_1 = values_adj_buffer.physical_address
my_ip.register_map.quantized_multiplier_offset_1 = quantized_multiplier_buffer.physical_address
my_ip.register_map.bias_offset_1  = bias_buffer.physical_address
my_ip.register_map.shift_offset_1  = shift_buffer.physical_address
my_ip.register_map.profiling_offset_1  = profiling_buffer.physical_address
my_ip.register_map.M_fea=M_fea
my_ip.register_map.N_adj=N_adj
my_ip.register_map.M_adj=N_adj
my_ip.register_map.P_w=P_w
my_ip.register_map.use_sblocks=0
my_ip.register_map.gemm_mode=0
my_ip.register_map.bias_count=0
my_ip.register_map.nnz_fea=NNZ_fea
my_ip.register_map.array_c_adjust=N_adj

In [878]:
def run_kernel():
    my_ip.register_map.CTRL.AP_START=1
    kernel_done = my_ip.register_map.CTRL.AP_DONE
    while kernel_done == 0:
            kernel_done = my_ip.register_map.CTRL.AP_DONE

In [879]:
## Start the IP and time its execution time
print("Running..")

Running..


In [880]:
%timeit run_kernel()
#run_kernel()

4.65 ms ± 2.58 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Check the contents of the memory to see if they look OK

In [79]:
result=np.zeros(shape=(N_adj,P_w), dtype=np.float16)
result[:] = D_buffer[:]
result[:] = result[:]

In [81]:
with np.printoptions(threshold=np.inf,suppress=True):
    print(result[0])

[0.001968 0.4778   0.5967   0.03113  0.002258 0.5645   0.5312   0.01105
 0.1146   0.716    0.0995   0.1425   0.653    0.1676   0.001312 0.07336
 0.12164  0.1553   0.6055   0.4922   0.4949  ]


In [40]:
result

array([[ 0.362  ,  0.4805 ,  0.4055 , ...,  0.3713 ,  0.497  ,  0.4531 ],
       [ 0.4678 ,  0.2988 ,  0.3374 , ...,  0.698  ,  0.2983 ,  0.3792 ],
       [-0.07153,  0.59   ,  0.0504 , ...,  0.609  ,  0.7686 ,  0.1338 ],
       ...,
       [ 0.502  ,  0.4756 ,  0.4128 , ...,  0.488  ,  0.4485 ,  0.363  ],
       [ 0.3262 ,  0.6333 ,  0.5176 , ...,  0.3574 ,  0.572  ,  0.3357 ],
       [ 0.496  ,  0.3784 ,  0.749  , ...,  0.1355 ,  0.392  ,  0.3357 ]],
      dtype=float16)

In [484]:
print("fifo_full_0 (frontend process fast)=",profiling_buffer[0])
print("fifo_empty_0 (frontend process slow)=",profiling_buffer[3])
print("fifo_read_0 (data read process count)=",profiling_buffer[6])
print("fifo_write_0 (data write process count)=",profiling_buffer[9])
print("fifo_cycle_0 (data activation process count)=",profiling_buffer[12])

fifo_full_0 (frontend process fast)= 0
fifo_empty_0 (frontend process slow)= 0
fifo_read_0 (data read process count)= 0
fifo_write_0 (data write process count)= 0
fifo_cycle_0 (data activation process count)= 0


In [485]:
print("fifo_full_2 (backend process slow)=",profiling_buffer[2])
print("fifo_empty_2 (backend process fast)=",profiling_buffer[5])
print("fifo_read_2 (data read process count)=",profiling_buffer[8])
print("fifo_write_2 (data write process count)=",profiling_buffer[11])
print("fifo_cycle_2 (data activation process count)=",profiling_buffer[14])

fifo_full_2 (backend process slow)= 0
fifo_empty_2 (backend process fast)= 0
fifo_read_2 (data read process count)= 0
fifo_write_2 (data write process count)= 0
fifo_cycle_2 (data activation process count)= 0


In [486]:
my_monitor.register_map


RegisterMap {
  reg0 = Register(value=4),
  reg1 = Register(value=0),
  reg2 = Register(value=0),
  reg3 = Register(value=0),
  reg4 = Register(value=0),
  reg5 = Register(value=0)
}

In [487]:
#my_ip.register_map.CTRL
fea_vfifo_full=int(my_monitor.register_map.reg1)
fea_vfifo_empty=int(my_monitor.register_map.reg2)
adj_vfifo_full=int(my_monitor.register_map.reg3)
adj_vfifo_empty=int(my_monitor.register_map.reg4)
active=int(my_monitor.register_map.reg0)

reg1/reg2 readval_fea (value fifo full/empty)
reg3/reg4 readval_adj (value fifo full/empty)
reg0 active

In [488]:
print("vfifo fea empty rate ",fea_vfifo_empty/active)

vfifo fea empty rate  0.0


In [440]:
print("vfifo fea full rate ",fea_vfifo_full/active)

vfifo fea full rate  0.4590431070922145


In [441]:
print("vfifo adj empty rate ",adj_vfifo_empty/active)

vfifo adj empty rate  0.49534405661931524


In [442]:
print("vfifo adj full rate ",adj_vfifo_full/active)

vfifo adj full rate  0.28641042840005615


In [443]:
print("Execution time in ms",1000*active/250000000)

Execution time in ms 10.513444


In [192]:
my_ip.register_map.CTRL.AUTO_RESTART=1

# Replicate in software to compare performance

for i in range(32):
    for j in range(32):
        A_buffer[i][j] = 1.0
for i in range(32):
    for j in range(32):
        B_buffer[i][j] = 1.0
for i in range(32):
    for j in range(32):
        C_buffer[i][j] = 0.0

%timeit C_buffer = A_buffer * B_buffer

C_buffer[0][1]

In [324]:
quantized_multiplier_buffer.freebuffer()
bias_buffer.freebuffer()
shift_buffer.freebuffer()
profiling_buffer.freebuffer()
rowPtr_fea_buffer.freebuffer()
columnIndex_fea_buffer.freebuffer()
values_fea_buffer.freebuffer()
rowPtr_adj_buffer.freebuffer()
columnIndex_adj_buffer.freebuffer()
values_adj_buffer.freebuffer()
B_buffer.freebuffer()
D_buffer.freebuffer()

In [325]:
from scipy.sparse import csr_matrix
csrMatrix_adj = csr_matrix((values_adj_row , columnIndex_adj_row, rowPtr_adj_row), shape=(N_adj, N_adj))
csrMatrix_fea = csr_matrix((values_fea_row , columnIndex_fea_row, rowPtr_fea_row), shape=(N_adj, M_fea))

In [439]:
w.shape

(500, 18)

In [328]:
%timeit csrMatrix_adjfeaw = csrMatrix_adj*(csrMatrix_fea*w)
#csrMatrix_adjfeaw = csrMatrix_adj*(csrMatrix_fea*w)

268 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
csrMatrix_adjfeaw.shape

(3327, 21)

In [101]:
with np.printoptions(threshold=np.inf,suppress=True):
    print(csrMatrix_adjfeaw[0])

[0.0019877  0.47759533 0.5971749  0.03115976 0.00219059 0.5643878
 0.530963   0.01102665 0.11454511 0.7159475  0.09950522 0.14244223
 0.6527748  0.16745949 0.00133669 0.073318   0.12169862 0.15521336
 0.60524035 0.4921999  0.4949317 ]


In [102]:
np.allclose(result, csrMatrix_adjfeaw,atol=0.1)

False

In [109]:
#support = torch.matmul(csrMatrix_fea,w)
#output = torch.spmm(csrMatrix_adj, support)
support = torch.matmul(csrMatrix_fea,w)

TypeError: matmul(): argument 'input' (position 1) must be Tensor, not csr_matrix